Analyzing Mobile App Data

In [1]:
from csv import reader

In [2]:
def reading(filename):
    open_file = open(filename,encoding='utf8')
    read_file = reader(open_file)
    return list(read_file)

def manage_column(dataset):
    return dataset[0],dataset[1:]

def explore_data(dataset,col_name,data_range=3):
    print("Column name")
    print(col_name)
    print("Dataset")
    for i in range(data_range):
        print(dataset[i])
    print(f"Number of rows {len(dataset)} Number of Column {len(col_name)}")

In [3]:
apple = reading("AppleStore.csv")
google = reading("googleplaystore.csv")

In [4]:
apps_data_col,apps_data = manage_column(apple)
google_data_col,google_data = manage_column(google)

In [5]:
explore_data(apps_data,apps_data_col,data_range=1)

Column name
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
Dataset
['281656475', 'PAC-MAN Premium', '100788224', 'USD', '3.99', '21292', '26', '4', '4.5', '6.3.5', '4+', 'Games', '38', '5', '10', '1']
Number of rows 7197 Number of Column 16


In [6]:
explore_data(google_data,google_data_col,data_range=1)

Column name
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
Dataset
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']
Number of rows 10841 Number of Column 13


Finding Missing Data on Google Play Dataset by Using Kaggle Discussion Forum 


https://www.kaggle.com/datasets/lava18/google-play-store-apps/discussion

In [7]:
missing_data_index = 10472
print(google_data[10472])

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


Find Missing Data on Google Play Dataset by Using Function

In [8]:
def find_missing(dataset):
    index = 0
    list_of_missing_data = []
    for row in dataset:
        if '' in row:
            list_of_missing_data.append(index)
        index += 1
    return list_of_missing_data

In [9]:
find_missing(google_data)

[1553, 10472]

In [10]:
#The missing data is on Current Ver column. Therefore, keeping this data should be fine
print(google_data[1553])

['Market Update Helper', 'LIBRARIES_AND_DEMO', '4.1', '20145', '11k', '1,000,000+', 'Free', '0', 'Everyone', 'Libraries & Demo', 'February 12, 2013', '', '1.5 and up']


In [11]:
#Deleting the missing data
del google_data[10472]
explore_data(google_data,google_data_col,data_range=1)

Column name
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
Dataset
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']
Number of rows 10840 Number of Column 13


Finding Number of Duplicated rows

In [12]:
def duplicate_count(dataset,name_index):
    duplicate_apps = []
    unique_apps = []
    for row in dataset:
        name = row[name_index]
        if name in unique_apps:
            duplicate_apps.append(name)
        else:
            unique_apps.append(name)
    print(f"Number of Duplicate Apps: {len(duplicate_apps)}")
    print(f"Example of Duplicate Apps: {duplicate_apps[:5]}")

In [13]:
duplicate_count(google_data,0)

Number of Duplicate Apps: 1181
Example of Duplicate Apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings']


In [14]:
#Google data should have 9659 rows
len(google_data) - 1181

9659

In [15]:
duplicate_count(apps_data,0)

Number of Duplicate Apps: 0
Example of Duplicate Apps: []


Removing Duplicate Entry

In [16]:
#Create a dictionary where key is a unique app name and the value is the highest number of reviews of that app
def name_with_highest_reviews(dataset,name_index,review_index):
    reviews_max = {}
    for row in dataset:
        name = row[name_index]
        n_reviews = float(row[review_index])
        if name in reviews_max and reviews_max[name] < n_reviews:
            reviews_max[name] = n_reviews
        if name not in reviews_max:
            reviews_max[name] = n_reviews
    return reviews_max

In [17]:
google_dupli_data = name_with_highest_reviews(google_data,0,3)
len(google_dupli_data)

9659

In [18]:
def remove_duplicate_rows(dataset,reviews_max,name_index,review_index):
    clean = []
    already_added = []
    for row in dataset:
        name = row[name_index]
        n_reviews = float(row[review_index])
        if (n_reviews == reviews_max[name]) and (name not in already_added):
            clean.append(row)
            already_added.append(name)
    return clean


In [19]:
clean_android_data = remove_duplicate_rows(google_data,google_dupli_data,0,3)

In [20]:
len(clean_android_data)

9659

Testing Removing None English Words

In [21]:
def eng_check(string):
    counter = 0
    for i in string:
        if ord(i) > 127:
            counter += 1
    if counter <= 3:
        return True
    return False

In [22]:
eng_check('Instagram')

True

In [23]:
eng_check('爱奇艺PPS -《欢乐颂2》电视剧热播')

False

In [24]:
eng_check('Docs To Go™ Free Office Suite')

True

In [25]:
eng_check('Instachat 😜')

True

Removing None English words in Dataset

In [26]:
def remove_non_eng(dataset,name_index):
    eng_list = []
    for row in dataset:
        name = row[name_index]
        if eng_check(name) == True:
            eng_list.append(row)
    return eng_list

In [27]:
goo_eng_clean_list = remove_non_eng(clean_android_data,0)

In [28]:
len(goo_eng_clean_list)

9614

In [29]:
app_eng_clean_list = remove_non_eng(apps_data,1)

In [30]:
len(app_eng_clean_list)

6183

In [31]:
explore_data(goo_eng_clean_list,google_data_col,data_range=3)

Column name
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
Dataset
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']
['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']
['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']
Number of rows 9614 Number of Column 13


In [32]:
explore_data(app_eng_clean_list,apps_data_col,data_range=3)

Column name
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
Dataset
['281656475', 'PAC-MAN Premium', '100788224', 'USD', '3.99', '21292', '26', '4', '4.5', '6.3.5', '4+', 'Games', '38', '5', '10', '1']
['281796108', 'Evernote - stay organized', '158578688', 'USD', '0', '161065', '26', '4', '3.5', '8.2.2', '4+', 'Productivity', '37', '5', '23', '1']
['281940292', 'WeatherBug - Local Weather, Radar, Maps, Alerts', '100524032', 'USD', '0', '188583', '2822', '3.5', '4.5', '5.0.0', '4+', 'Weather', '37', '5', '3', '1']
Number of rows 6183 Number of Column 16


Isolating the Free Apps

In [33]:
def free_application(dataset,free_index):
    free_data = []
    for row in dataset:
        if row[free_index] == '0':
            free_data.append(row)
    return free_data

In [34]:
app_eng_clean_free_list = free_application(app_eng_clean_list,4)

In [35]:
len(app_eng_clean_free_list)

3222

In [36]:
goo_eng_clean_free_list = free_application(goo_eng_clean_list,7)

In [37]:
len(goo_eng_clean_free_list)

8864

Genre Counting

In [38]:
def genre_count(data_set,genre_index):
    genre_dict = {}
    for row in data_set:
        genre_name = row[genre_index]
        if genre_name in genre_dict:
            genre_dict[genre_name] += 1
        else:
            genre_dict[genre_name] = 1
    return genre_dict

In [39]:
apple_common_genre = genre_count(app_eng_clean_free_list,11)

In [40]:
apple_common_genre

{'Productivity': 56,
 'Weather': 28,
 'Shopping': 84,
 'Reference': 18,
 'Finance': 36,
 'Music': 66,
 'Utilities': 81,
 'Travel': 40,
 'Social Networking': 106,
 'Sports': 69,
 'Health & Fitness': 65,
 'Games': 1874,
 'Food & Drink': 26,
 'News': 43,
 'Book': 14,
 'Photo & Video': 160,
 'Entertainment': 254,
 'Business': 17,
 'Lifestyle': 51,
 'Education': 118,
 'Navigation': 6,
 'Medical': 6,
 'Catalogs': 4}

In [41]:
google_common_genre = genre_count(goo_eng_clean_free_list,9)

In [42]:
google_common_genre

{'Art & Design': 53,
 'Art & Design;Creativity': 6,
 'Auto & Vehicles': 82,
 'Beauty': 53,
 'Books & Reference': 190,
 'Business': 407,
 'Comics': 54,
 'Comics;Creativity': 1,
 'Communication': 287,
 'Dating': 165,
 'Education': 474,
 'Education;Creativity': 4,
 'Education;Education': 30,
 'Education;Pretend Play': 5,
 'Education;Brain Games': 3,
 'Entertainment': 538,
 'Entertainment;Brain Games': 7,
 'Entertainment;Creativity': 3,
 'Entertainment;Music & Video': 15,
 'Events': 63,
 'Finance': 328,
 'Food & Drink': 110,
 'Health & Fitness': 273,
 'House & Home': 73,
 'Libraries & Demo': 83,
 'Lifestyle': 345,
 'Lifestyle;Pretend Play': 1,
 'Card': 40,
 'Arcade': 164,
 'Puzzle': 100,
 'Racing': 88,
 'Sports': 307,
 'Casual': 156,
 'Simulation': 181,
 'Adventure': 60,
 'Trivia': 37,
 'Action': 275,
 'Word': 23,
 'Role Playing': 83,
 'Strategy': 81,
 'Board': 34,
 'Music': 18,
 'Action;Action & Adventure': 9,
 'Casual;Brain Games': 12,
 'Educational;Creativity': 3,
 'Puzzle;Brain Games':

In [43]:
google_common_category = genre_count(goo_eng_clean_free_list,1)

In [44]:
google_common_category

{'ART_AND_DESIGN': 57,
 'AUTO_AND_VEHICLES': 82,
 'BEAUTY': 53,
 'BOOKS_AND_REFERENCE': 190,
 'BUSINESS': 407,
 'COMICS': 55,
 'COMMUNICATION': 287,
 'DATING': 165,
 'EDUCATION': 103,
 'ENTERTAINMENT': 85,
 'EVENTS': 63,
 'FINANCE': 328,
 'FOOD_AND_DRINK': 110,
 'HEALTH_AND_FITNESS': 273,
 'HOUSE_AND_HOME': 73,
 'LIBRARIES_AND_DEMO': 83,
 'LIFESTYLE': 346,
 'GAME': 862,
 'FAMILY': 1676,
 'MEDICAL': 313,
 'SOCIAL': 236,
 'SHOPPING': 199,
 'PHOTOGRAPHY': 261,
 'SPORTS': 301,
 'TRAVEL_AND_LOCAL': 207,
 'TOOLS': 750,
 'PERSONALIZATION': 294,
 'PRODUCTIVITY': 345,
 'PARENTING': 58,
 'WEATHER': 71,
 'VIDEO_PLAYERS': 159,
 'NEWS_AND_MAGAZINES': 248,
 'MAPS_AND_NAVIGATION': 124}

In [45]:
def display_table(dataset):
    display_list = []
    for key in dataset:
        display_list.append((round(dataset[key],2),key))

    return sorted(display_list,reverse= True)

In [46]:
display_table(google_common_genre)

[(749, 'Tools'),
 (538, 'Entertainment'),
 (474, 'Education'),
 (407, 'Business'),
 (345, 'Productivity'),
 (345, 'Lifestyle'),
 (328, 'Finance'),
 (313, 'Medical'),
 (307, 'Sports'),
 (294, 'Personalization'),
 (287, 'Communication'),
 (275, 'Action'),
 (273, 'Health & Fitness'),
 (261, 'Photography'),
 (248, 'News & Magazines'),
 (236, 'Social'),
 (206, 'Travel & Local'),
 (199, 'Shopping'),
 (190, 'Books & Reference'),
 (181, 'Simulation'),
 (165, 'Dating'),
 (164, 'Arcade'),
 (157, 'Video Players & Editors'),
 (156, 'Casual'),
 (124, 'Maps & Navigation'),
 (110, 'Food & Drink'),
 (100, 'Puzzle'),
 (88, 'Racing'),
 (83, 'Role Playing'),
 (83, 'Libraries & Demo'),
 (82, 'Auto & Vehicles'),
 (81, 'Strategy'),
 (73, 'House & Home'),
 (71, 'Weather'),
 (63, 'Events'),
 (60, 'Adventure'),
 (54, 'Comics'),
 (53, 'Beauty'),
 (53, 'Art & Design'),
 (44, 'Parenting'),
 (40, 'Card'),
 (38, 'Casino'),
 (37, 'Trivia'),
 (35, 'Educational;Education'),
 (34, 'Board'),
 (33, 'Educational'),
 (30, 

In [47]:
display_table(apple_common_genre)

[(1874, 'Games'),
 (254, 'Entertainment'),
 (160, 'Photo & Video'),
 (118, 'Education'),
 (106, 'Social Networking'),
 (84, 'Shopping'),
 (81, 'Utilities'),
 (69, 'Sports'),
 (66, 'Music'),
 (65, 'Health & Fitness'),
 (56, 'Productivity'),
 (51, 'Lifestyle'),
 (43, 'News'),
 (40, 'Travel'),
 (36, 'Finance'),
 (28, 'Weather'),
 (26, 'Food & Drink'),
 (18, 'Reference'),
 (17, 'Business'),
 (14, 'Book'),
 (6, 'Navigation'),
 (6, 'Medical'),
 (4, 'Catalogs')]

In [48]:
display_table(google_common_category)

[(1676, 'FAMILY'),
 (862, 'GAME'),
 (750, 'TOOLS'),
 (407, 'BUSINESS'),
 (346, 'LIFESTYLE'),
 (345, 'PRODUCTIVITY'),
 (328, 'FINANCE'),
 (313, 'MEDICAL'),
 (301, 'SPORTS'),
 (294, 'PERSONALIZATION'),
 (287, 'COMMUNICATION'),
 (273, 'HEALTH_AND_FITNESS'),
 (261, 'PHOTOGRAPHY'),
 (248, 'NEWS_AND_MAGAZINES'),
 (236, 'SOCIAL'),
 (207, 'TRAVEL_AND_LOCAL'),
 (199, 'SHOPPING'),
 (190, 'BOOKS_AND_REFERENCE'),
 (165, 'DATING'),
 (159, 'VIDEO_PLAYERS'),
 (124, 'MAPS_AND_NAVIGATION'),
 (110, 'FOOD_AND_DRINK'),
 (103, 'EDUCATION'),
 (85, 'ENTERTAINMENT'),
 (83, 'LIBRARIES_AND_DEMO'),
 (82, 'AUTO_AND_VEHICLES'),
 (73, 'HOUSE_AND_HOME'),
 (71, 'WEATHER'),
 (63, 'EVENTS'),
 (58, 'PARENTING'),
 (57, 'ART_AND_DESIGN'),
 (55, 'COMICS'),
 (53, 'BEAUTY')]

Finding Most Popular Application in Appstore

In [49]:
def most_popular(col,dataset,genre_index,user_index):
    popular_dict = {}
    for genre in col:
        total = 0
        len_genre = 0
        avg_rating = 0
        for row in dataset:
            genre_app = row[genre_index]
            if genre_app == genre:
                user_rating = row[user_index]
                user_rating = user_rating.replace(',', '')
                user_rating = user_rating.replace('+', '')
                total += float(user_rating)
                len_genre += 1
        avg_rating = total / len_genre
        popular_dict[genre] = avg_rating
    return popular_dict

In [50]:
most_popular_apple = most_popular(apple_common_genre,app_eng_clean_free_list,11,5)

In [51]:
display_table(most_popular_apple)

[(86090.33, 'Navigation'),
 (74942.11, 'Reference'),
 (71548.35, 'Social Networking'),
 (57326.53, 'Music'),
 (52279.89, 'Weather'),
 (39758.5, 'Book'),
 (33333.92, 'Food & Drink'),
 (31467.94, 'Finance'),
 (28441.54, 'Photo & Video'),
 (28243.8, 'Travel'),
 (26919.69, 'Shopping'),
 (23298.02, 'Health & Fitness'),
 (23008.9, 'Sports'),
 (22788.67, 'Games'),
 (21248.02, 'News'),
 (21028.41, 'Productivity'),
 (18684.46, 'Utilities'),
 (16485.76, 'Lifestyle'),
 (14029.83, 'Entertainment'),
 (7491.12, 'Business'),
 (7003.98, 'Education'),
 (4004.0, 'Catalogs'),
 (612.0, 'Medical')]

In [52]:
most_popular_android = most_popular(google_common_category,goo_eng_clean_free_list,1,5)

In [53]:
display_table(most_popular_android)

[(38456119.17, 'COMMUNICATION'),
 (24727872.45, 'VIDEO_PLAYERS'),
 (23253652.13, 'SOCIAL'),
 (17840110.4, 'PHOTOGRAPHY'),
 (16787331.34, 'PRODUCTIVITY'),
 (15588015.6, 'GAME'),
 (13984077.71, 'TRAVEL_AND_LOCAL'),
 (11640705.88, 'ENTERTAINMENT'),
 (10801391.3, 'TOOLS'),
 (9549178.47, 'NEWS_AND_MAGAZINES'),
 (8767811.89, 'BOOKS_AND_REFERENCE'),
 (7036877.31, 'SHOPPING'),
 (5201482.61, 'PERSONALIZATION'),
 (5074486.2, 'WEATHER'),
 (4188821.99, 'HEALTH_AND_FITNESS'),
 (4056941.77, 'MAPS_AND_NAVIGATION'),
 (3695641.82, 'FAMILY'),
 (3638640.14, 'SPORTS'),
 (1986335.09, 'ART_AND_DESIGN'),
 (1924897.74, 'FOOD_AND_DRINK'),
 (1833495.15, 'EDUCATION'),
 (1712290.15, 'BUSINESS'),
 (1437816.27, 'LIFESTYLE'),
 (1387692.48, 'FINANCE'),
 (1331540.56, 'HOUSE_AND_HOME'),
 (854028.83, 'DATING'),
 (817657.27, 'COMICS'),
 (647317.82, 'AUTO_AND_VEHICLES'),
 (638503.73, 'LIBRARIES_AND_DEMO'),
 (542603.62, 'PARENTING'),
 (513151.89, 'BEAUTY'),
 (253542.22, 'EVENTS'),
 (120550.62, 'MEDICAL')]